PREPROCESSING

In [1]:
import pandas as pd
import numpy as np

fields=['UMUR','JENIS_KELAMIN', 'FOTO_TORAKS','STATUS_HIV',
'RIWAYAT_DIABETES','LOKASI_ANATOMI']
dataset=pd.read_csv('toy50.csv',usecols=fields,delimiter=";")

#definisi atribut
umur=dataset.UMUR.values
jenis_kelamin=dataset.JENIS_KELAMIN.values
foto_toraks=dataset.FOTO_TORAKS.values
status_hiv=dataset.STATUS_HIV.values
riwayat_diabetes=dataset.RIWAYAT_DIABETES.values
target=dataset.LOKASI_ANATOMI.values

jumlahdata=len(dataset)
umur_max = max(umur)
umur_min = min(umur)
datanew=[]
for i in range(jumlahdata):
    labelumur= 0 if(umur[i]<=5) else 1 if(umur[i]>= 6 and umur[i]<=11) else 2 if(umur[i]>= 12 and umur[i]<=16) else 3 if(umur[i]>= 17 and umur[i]<=25) else 4 if(umur[i]>= 26 and umur[i]<=35) else 5 if(umur[i]>= 36 and umur[i]<=45) else 6 if(umur[i]>= 46 and umur[i]<=55) else 7 if(umur[i]>= 56 and umur[i]<=65) else 8 if(umur[i]>=66) else None
    labeljk= 1 if(jenis_kelamin[i]=="L") else 0 
    labelft= 1 if(foto_toraks[i]=="Positif") else 0 if(foto_toraks[i]=="Negatif") else None
    labelsh= 1 if(status_hiv[i]=="Positif") else 0 if(status_hiv[i]=="Negatif") else None
    labelrd= 1 if(riwayat_diabetes[i]=="Ya") else 0 if(riwayat_diabetes[i]=="Tidak") else None
    # labeltcm= 2 if(hasil_tcm[i]=="Rif Resisten") else 1 if(hasil_tcm[i]=="Rif Sensitif") else 0 if(hasil_tcm[i]=="Negatif") else None
    labeltarget= 0 if(target[i]=="Paru") else 1 if(target[i]=="Ekstra Paru") else 1
    datanew.append([labelumur,labeljk,labelft,labelsh,labelrd,labeltarget])
udtf= pd.DataFrame(datanew,columns=fields)
# print(udtf)
udtf.to_csv('D:/KULIAH/SKRIPSI/SIDANG/normalisasi_toy15.csv')

MISSING VALUE KNN

In [2]:
from math import sqrt
from re import L
import pandas as pd
import numpy as np
import statistics
from statistics import mode

fields=['UMUR','JENIS_KELAMIN', 'FOTO_TORAKS','STATUS_HIV',
'RIWAYAT_DIABETES','LOKASI_ANATOMI']
datasetNorm=pd.read_csv('normalisasi_toy50.csv',usecols=fields)

#definisi atribut
umur=datasetNorm.UMUR.values
jenis_kelamin=datasetNorm.JENIS_KELAMIN.values
foto_toraks=datasetNorm.FOTO_TORAKS.values
status_hiv=datasetNorm.STATUS_HIV.values
riwayat_diabetes=datasetNorm.RIWAYAT_DIABETES.values
lokasi_anatomi=datasetNorm.LOKASI_ANATOMI.values

In [3]:
banyak_data = len(datasetNorm)
# print(banyak_data)
# sum_mv = datasetNorm.isna().sum() 
# print("======MISSING VALUE======\n",sum_mv)

k=9 
missingData, fullData=[],[] #menyimpan index dari record data kosong & index dari record data penuh
missKol_List=[]
e_dis=[]
indexSortAll,neighborAll=[],[]

def getData(row):
    rowData = []
    getDataRow = datasetNorm.iloc[row].values
    for i in getDataRow:
        rowData.append(i)
    return rowData

def getFullData(fullData):
    for i in range(banyak_data):
        if(pd.notnull(getData(i)[0]) and pd.notnull(getData(i)[1]) and pd.notnull(getData(i)[2])
        and pd.notnull(getData(i)[3]) and pd.notnull(getData(i)[4])):
            fullData.append(i)
        else:
            None
    print("FUll",fullData)
    fullFrame= pd.DataFrame(getData(fullData),columns=fields)
    # print(fullFrame)
    fullFrame.to_csv('D:/KULIAH/SKRIPSI/SIDANG/fullFrame_toy50.csv')
    print("\nJUMLAH DATA RECORD NOT MISSING:",len(fullData))

def getMissingData(missingData):
    for i in range(banyak_data):
        if(i not in fullData):
            missingData.append(i)
    # print("MISSING",missingData)
    missingFrame= pd.DataFrame(getData(missingData),columns=fields)
    # print(missingFrame)
    missingFrame.to_csv('D:/KULIAH/SKRIPSI/SIDANG/missingFrame_toy50.csv')
    print("JUMLAH DATA RECORD MISSING:",len(missingData))
    for i in missingData:
        for j in range(len(getData(i))):
            if(pd.isnull(getData(i)[j])):
                missKol_List.append([i,j])
    print("MISSING IN COLOUMN",missKol_List,"\n")

def euclideanStep():
    #EUCLIDEAN
    for i in range(len(missKol_List)):
        missingLine=missKol_List[i][0]
        dis_kol =[]
        for j in range(len(fullData)):
            fullLine=fullData[j]
            # print(i,"missing point:",getData(missingLine),">>>other point:",getData(fullLine))
            numRow = len(getData(missingLine)) #panjang fitur/panjang atribut
            euc=0
            # for x in range(numRow):
            for x in range(numRow-1):
                if(pd.notnull(getData(missingLine)[x]) and pd.notnull(getData(fullLine)[x])):
                    euc+=(getData(missingLine)[x] - getData(fullLine)[x]) * (getData(missingLine)[x] - getData(fullLine)[x])
            eucValue=sqrt(euc)
            dis_kol.append(eucValue)
        e_dis.append(dis_kol)
        indexSort=np.argsort(e_dis[i])[:k]
        indexSortAll.append(indexSort)
    
def neighborStep():
    for x in range(len(indexSortAll)):
        neighbor=[]
        for y in range(len(indexSortAll[x])):
            indexNeighbor=indexSortAll[x][y]
            neighbor.append(fullData[indexNeighbor])
        neighborAll.append(neighbor)
    # print(neighborAll,"\n")

toReplace=[]
def mostFrequentStep():
    for x in range(len(missKol_List)):
        neighborLabel=[]
        for y in range(len(neighborAll[x])):
            missingCol=missKol_List[x][1]
            neighborCol=neighborAll[x][y]
            neighborValue=getData(neighborCol)[missingCol]
            # print(x,"KOL",missingCol,"neighbor",neighborCol,">>>value",neighborValue)
            neighborLabel.append(neighborValue)
        mostFrequent=mode(neighborLabel)
        # print("missing index baris",missKol_List[x][0],"index kolom",missingCol,">>>most frequent",mostFrequent)
        toReplace.append([missKol_List[x][0],missingCol,mostFrequent])
    # print(toReplace)

changed_data = []
def imputationStep():
    for i in range(banyak_data):
        baris = getData(i)
        # print(i,baris)
        for j in range(len(baris)):
            for x in range(len(toReplace)):
                if(i == toReplace[x][0] and j == toReplace[x][1]):
                    baris[j]=toReplace[x][2]
        # print(i,'>>>changed',baris)
        changed_data.append(baris)
    # print('clean data :',changed_data)

def toDataFrame():
    updateToy50_knn= pd.DataFrame(changed_data,columns=fields)
    print(updateToy50_knn)
    updateToy50_knn.to_csv('D:/KULIAH/SKRIPSI/SIDANG/updateToy50_knn.csv')


full=getFullData(fullData)
missing=getMissingData(missingData)
euc=euclideanStep()
neighbor=neighborStep()
mostFrequent=mostFrequentStep()
imputation=imputationStep()
dframe=toDataFrame()

   

FUll [0, 1, 2, 3, 5, 7, 8, 9, 10, 11, 13, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 40, 42, 47, 48, 49]

JUMLAH DATA RECORD NOT MISSING: 37
JUMLAH DATA RECORD MISSING: 13
MISSING IN COLOUMN [[4, 2], [6, 2], [12, 3], [14, 2], [17, 3], [26, 2], [38, 2], [39, 2], [39, 4], [41, 2], [43, 2], [44, 2], [45, 2], [46, 2]] 

    UMUR  JENIS_KELAMIN  FOTO_TORAKS  STATUS_HIV  RIWAYAT_DIABETES  \
0    3.0            1.0          1.0         0.0               0.0   
1    3.0            1.0          1.0         0.0               0.0   
2    5.0            1.0          1.0         0.0               0.0   
3    7.0            1.0          1.0         0.0               0.0   
4    3.0            1.0          1.0         0.0               0.0   
5    7.0            1.0          1.0         0.0               0.0   
6    4.0            1.0          1.0         0.0               0.0   
7    4.0            1.0          1.0         0.0               0.0   
8    6.0   